The Black-Scholes is a mathematical method to model the dynamics of a derivative instrument. Its formula estimates the theoretical value of European-style options and shows that the option has a unique price given the security and its expected return.

In [50]:
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm
from pandas_datareader.data import DataReader

Loosely defined, the Black-Scholes call option price $C$ is calculated by subtracting the present value of the strike price $K$ multiplied by the risk-adjusted probability that the option will be exercised $N(d_2)$, of the product of the stock price $S$ by the probability of receiving the stock at the expiration of the option contract $N(d_1)$.

$$C = SN(d_1) - K e^{-rt} N(d_2)$$

$$d_1 = \frac{ln_K^S + (r+ \frac{\sigma^2}{2} \Delta t) }{\sigma * \sqrt{\Delta t}}$$

$$d_2 = d_1 - \sigma \sqrt{\Delta t}$$

where:
1. $C$ - Call option price
2. $S$ - Current stock price (or other underlying instrument)
3. $K$ - Strike price
4. $r$ - Risk-free interest rate
5. $\Delta t$ - Time to maturity (T-t)
6. $N$ - Normal distribution
7. $\sigma$ - Volatility of the asset
8. $N(d_1)$ - Probability of receiving the stock at the expiration of our option contract. (The expected value of the receipt of the stock is contingent on the exercise of the option. It is, therefore, the product of the conditional expected value of the receipt of the stock given that exercise has occurred times the probability of exercise)
9. $N(d_2)$ - Risk-adjusted probability that the option will be exercised

Assumptions:
1. Constant volatility is assumed, although this may not be the case. Replacing this term with stochastic-process generated volatility would reduce error.
2. Returns are assumed to be normally distributed.
3. Risk-free rate is hypothetical assuming that all payment obligations are met.
4. The underlying stock does not pay dividends during the option contract. This can be adjusted by subtracting the discounted value of a future dividend from the stock price.
5. Assumes that markets function efficiently and are perfectly liquid, with the stock price moving independently and possibility to buy or sell any amount at any given time.

In [47]:
def d1(S, K, r, dt, sigma):
    return (np.log(S/K) + (r + np.sqrt(sigma) * .5 * dt)) / (sigma * np.sqrt(dt))

def d2(S, K, r, dt, sigma):
    return d1(S, K, r, dt, sigma) - sigma * np.sqrt(dt)

def call(S, K, r, dt, sigma):
    return S * norm.cdf(d1(S, K, r, dt, sigma)) - K * np.exp(-r * dt) * norm.cdf(d2(S, K, r, dt, sigma))

In the below data cell, we are querying price data for 1 year for a sample stock in our case Tesla from Yahoo Finance. And we set the risk-free rate to a 30 year U.S. treasury yield:

In [71]:
date = datetime.datetime.now()
date_m1 = date - datetime.timedelta(365)

df = DataReader('TSLA', 'yahoo', date_m1, date).sort_values('Date')
df['rel_day_returns'] = (df['Close'] - df['Close'].shift(1)) / df['Close'].shift(1)
display(df)

trsry30_yield = DataReader('^TNX', 'yahoo', date - datetime.timedelta(1), date)['Close'].iloc[0]
display(trsry30_yield)

,High,Low,Open,Close,Volume,Adj Close,rel_day_returns
Date,,,,,,,
2021-09-30,263.043335,258.333344,260.333344,258.493347,53868000.0,258.493347,NaN
2021-10-01,260.260010,254.529999,259.466675,258.406677,51094200.0,258.406677,-0.000335
2021-10-04,268.989990,258.706665,265.500000,260.510010,91449900.0,260.510010,0.008140
2021-10-05,265.769989,258.066681,261.600006,260.196655,55297800.0,260.196655,-0.001203
2021-10-06,262.220001,257.739990,258.733337,260.916656,43898400.0,260.916656,0.002767
...,...,...,...,...,...,...,...
2022-09-23,284.500000,272.820007,283.089996,275.329987,63615400.0,275.329987,-0.045948
2022-09-26,284.089996,270.309998,271.829987,276.010010,58076900.0,276.010010,0.002470
2022-09-27,288.670013,277.510010,283.839996,282.940002,61925200.0,282.940002,0.025108


3.747000217437744

In [81]:
S = df['Close'].iloc[-1]
K = 270
r = trsry30_yield / 100.
dt = 180 / 365
sigma = np.sqrt(df.index.unique().shape[0]) * df['rel_day_returns'].std()

In [83]:
C = call(S, K, r, dt, sigma)
C

47.86394239304241